In [0]:
#!Author :- Dibyanayan Bandyopadhyay


In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
import pandas as pd
file = '/content/gdrive/My Drive/multilingual grounded/final.csv'
df = pd.read_csv(file)

In [0]:
labels = list(df['gold_label'])
labs = []
for i in labels:
    if i=='contradiction':
        labs.append(0)
        
    elif i=='neutral':
        labs.append(1)
    elif i== 'entailment':
        labs.append(2)
    else:
        labs.append(3)
        


In [4]:
import numpy as np
%tensorflow_version 1.x
import tensorflow as tf
tf.enable_eager_execution()

TensorFlow 1.x selected.


In [0]:
#sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))

#tf.keras.backend.set_session(sess)
tf.debugging.set_log_device_placement(True)


In [0]:
lab = tf.keras.utils.to_categorical(labs)

In [0]:
premises,premises_hindi,hypos_hindi,hypos  = list(df['premise']),list(df['premise_hindi']),list(df['hypo_hindi']),list(df['hypo'])

In [0]:
sen1,sen2,sen3,sen4 = [],[],[],[]

for i in premises:
    sen1.append(tf.keras.preprocessing.text.text_to_word_sequence(i))
    
for i in hypos:
    sen2.append(tf.keras.preprocessing.text.text_to_word_sequence(i))
    
for i in premises_hindi:
    sen3.append(tf.keras.preprocessing.text.text_to_word_sequence(i))
    
for i in hypos_hindi:
    sen4.append(tf.keras.preprocessing.text.text_to_word_sequence(i))
    


In [9]:
max_len_prem = 0 #...................... max length of english premise sentence
for i in sen1:
    if len(i)>max_len_prem:
        max_len_prem = len(i)
print(max_len_prem)

max_len_hypo = 0 #....................... max length of english hypothesis sentence
for i in sen2:
    if len(i)>max_len_hypo:
        max_len_hypo = len(i)
print(max_len_hypo)

max_len_prem_ = 0 # max length of hindi premise sentences
for i in sen3:
    if len(i)>max_len_prem_:
        max_len_prem_ = len(i)
print(max_len_prem_)

max_len_hypo_ = 0 # max length of hindi hypothesis sentences
for i in sen4:
    if len(i)>max_len_hypo_:
        max_len_hypo_ = len(i)
print(max_len_hypo_)

78
39
95
47


In [0]:
premises = sen1 #english prem
hypos = sen2 #english hypo
premises_ = sen3 #hindi premises
hypos_ = sen4 # hindi hypos

for sentences in range(len(premises)):
    #print(sentences)
    length = len(premises[sentences])
    max_len = max_len_prem
    diff = max_len-length
    for i in range(diff):
        premises[sentences].append('nan') # appending 'nan' token to english premises to make length of every sentences same

for sentences in range(len(hypos)):
    #print(sentences)
    length = len(hypos[sentences])
    max_len = max_len_hypo
    diff = max_len-length
    for i in range(diff):
        hypos[sentences].append('nan')  # appending 'nan' token to english hypos to make length of every sentences same

for sentences in range(len(premises_)):
    #print(sentences)
    length = len(premises_[sentences])
    max_len = max_len_prem_
    diff = max_len-length
    for i in range(diff):
        premises_[sentences].append('nan')  # appending 'nan' token to hindi premises to make length of every sentences same

for sentences in range(len(hypos_)):
    #print(sentences)
    length = len(hypos_[sentences])
    max_len = max_len_hypo_
    diff = max_len-length
    for i in range(diff):
        hypos_[sentences].append('nan')  # appending 'nan' token to hindi hypos to make length of every sentences same

In [0]:

# generating padded docs to input in subsequent embedding layer
def generate_padded_docs(sentences):
    docs = sentences
  
    tok  = tf.keras.preprocessing.text.Tokenizer()
    tok.fit_on_texts(docs)
    docs = tok.texts_to_sequences(docs)
    #print(docs)
    max_len_multi = 0
    for i in docs:
        if len(i)>max_len_multi:
            max_len_multi = len(i)
    #print(max_len_multi)
    padded_docs = tf.keras.preprocessing.sequence.pad_sequences(docs, maxlen=max_len_multi, padding='post')
    vocab_size = max([max(i) for i in padded_docs])
    return padded_docs,max_len_multi,vocab_size

In [0]:
#generating padded docs from english premises and hypothesis
padded_docs_prem,max_len_multi_prem,vocab_size_prem = generate_padded_docs(premises)
padded_docs_hypo,max_len_multi_hypo,vocab_size_hypo = generate_padded_docs(hypos)

padded_docs_prem = np.asarray(padded_docs_prem,dtype = np.int32)
padded_docs_hypo = np.asarray(padded_docs_hypo,dtype = np.int32)


In [0]:
# generating padded docs from hindi premises and hypothesis
padded_docs_prem_,max_len_multi_prem_,vocab_size_prem_ = generate_padded_docs(premises_)
padded_docs_hypo_,max_len_multi_hypo_,vocab_size_hypo_ = generate_padded_docs(hypos_)

padded_docs_prem_ = np.asarray(padded_docs_prem_,dtype = np.int32)
padded_docs_hypo_ = np.asarray(padded_docs_hypo_,dtype = np.int32)

In [14]:
# this keras model will be used to train english and hindi model using embedding generated above (no w2v)  and without other language's contxt

strategy = tf.distribute.MirroredStrategy()
with strategy.scope():
  class MyModel1(tf.keras.Model):
      def __init__(self,word_vector_length,max_len_multi_prem,max_len_multi_hypo,vocab_size_prem,vocab_size_hypo):
          super(MyModel1, self).__init__()
          self.lstm_shared = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(40,return_sequences = False,kernel_initializer = tf.keras.initializers.lecun_normal(seed=None),unit_forget_bias = True, name = 'lstm_main_1'),name = 'bid_1',merge_mode = 'mul')
          self.d1 = tf.keras.layers.Dense(10, activation='relu')
          self.d2 = tf.keras.layers.Dense(4, activation='softmax')
      
          self.word_vector_length = word_vector_length
          self.max_len_multi_prem = max_len_multi_prem
          self.max_len_multi_hypo = max_len_multi_hypo
          self.vocab_size_prem = vocab_size_prem
          self.vocab_size_hypo = vocab_size_hypo
          self.embedding1 = tf.keras.layers.Embedding(self.vocab_size_prem+1, self.word_vector_length, input_length=self.max_len_multi_prem)
          self.embedding2 = tf.keras.layers.Embedding(self.vocab_size_hypo+1, self.word_vector_length ,input_length=self.max_len_multi_hypo)
      def call(self,inputs):
        
          h1 = self.embedding1(inputs[0])
          x= self.lstm_shared(h1)
          #h1 = tf.nn.embedding_lookup(self.word_embeddings, inputs[1])#inputs[1] is padded_docs
          h2 = self.embedding2(inputs[1])
          print(np.shape(h1))
          
          y = self.lstm_shared(h2)
          z = tf.math.abs(x-y)

          
          z = self.d1(z)
          return self.d2(z)
    # training on first 35k data of english language
  model1 = MyModel1(30,max_len_multi_prem,max_len_multi_hypo,vocab_size_prem,vocab_size_hypo)
  model1.compile(optimizer='adam',
                loss='categorical_crossentropy',
                metrics=['accuracy'])
  model1.fit([padded_docs_prem[0:35000,:],padded_docs_hypo[0:35000,:]],[np.asarray(lab[0:35000,:],dtype=np.int32)],epochs = 2)

Executing op RandomUniform in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Sub in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Mul in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Add in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarIsInitializedOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op LogicalNot in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Assert in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op TruncatedNormal in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op RandomStandardNormal in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Qr in device /job:localhost/re

In [16]:
x = model1.lstm_shared(model1.embedding1(padded_docs_prem))
y = model1.lstm_shared(model1.embedding2(padded_docs_hypo))
z = tf.math.abs(x-y)
print(z)
#print(np.sum(z_,axis = 1))
#print(np.sum(tf.keras.activations.softmax(z_),axis = 1))
z_ = model1.d1(z)
fin = model1.d2(z_)
from sklearn.metrics import accuracy_score
print(accuracy_score(np.argmax(fin[35000:,:],axis = 1),np.argmax(lab[35000:,:],axis = 1)))

tf.Tensor(
[[0.86744726 0.6140226  0.01665686 ... 0.96878815 0.27091035 0.52119684]
 [0.50891733 0.4197212  0.01676336 ... 0.92155373 0.22085156 0.64875454]
 [0.30674678 0.06245703 0.01667597 ... 0.89776754 0.1934443  0.8124353 ]
 ...
 [0.29280323 0.0628469  0.01145307 ... 0.75601965 0.24875158 0.90408695]
 [0.4008206  0.4809546  0.01077517 ... 0.86266816 0.16503544 0.58873546]
 [0.36267212 0.36395168 0.01794727 ... 0.91692054 0.17945114 0.72879636]], shape=(39998, 40), dtype=float32)
Executing op MatMul in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op BiasAdd in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Relu in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Softmax in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op StridedSlice in device /job:localhost/replica:0/task:0/device:GPU:0
0.5960384153661464


In [0]:
context = z
word_vector_length = 30

In [18]:
input1 = tf.keras.layers.Input(shape=(max_len_multi_prem_,), name='input1')
input2 = tf.keras.layers.Input(shape=(max_len_multi_hypo_,), name='input2')
embedding1 = tf.keras.layers.Embedding(vocab_size_prem_+1, word_vector_length, input_length=max_len_multi_prem_)(input1)
embedding2 = tf.keras.layers.Embedding(vocab_size_hypo_+1, word_vector_length ,input_length=max_len_multi_hypo_)(input2)
lstm_shared = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(40,return_sequences = False,kernel_initializer = tf.keras.initializers.lecun_normal(seed=None),unit_forget_bias = True, name = 'lstm_main_1'),name = 'bid_1',merge_mode = 'mul')
h1 = lstm_shared(embedding1)
h2 = lstm_shared(embedding2)
z = tf.math.abs(h1-h2)
aux_op = tf.keras.layers.Softmax(name = 'aux_output')(z)
d1 = tf.keras.layers.Dense(10, activation='relu')(z)
main_op = tf.keras.layers.Dense(4, activation='softmax',name = 'main_output')(d1)

Executing op RandomUniform in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Sub in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Mul in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Add in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op VarIsInitializedOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op LogicalNot in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Assert in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op TruncatedNormal in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op LogicalNot in device /job:localhost/rep

In [19]:
model_context = tf.keras.Model(inputs=[input1, input2], outputs=[main_op, aux_op])
model_context.compile(optimizer='adam',
              loss={'main_output': 'binary_crossentropy', 'aux_output': tf.keras.losses.KLDivergence()},
              loss_weights={'main_output': 1., 'aux_output': 0.2})

# And trained it via:
model_context.fit({'input1':padded_docs_prem_[0:35000,:], 'input2': padded_docs_hypo_[0:35000,:]},
          {'main_output': lab[0:35000,:], 'aux_output' : np.asarray(tf.keras.activations.softmax(context[0:35000,:],axis = 1),dtype = np.float32)},
          epochs=2, batch_size=32)

Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op StridedSlice in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op MapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op MapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op FlatMapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op TensorDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op RepeatDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ZipDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ParallelMapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Train on 35000 samples
Epoch 1/2
Executing op ModelDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op AnonymousIteratorV2 in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op MakeIterator in device /job:localhost/replica:0/task:

In [20]:
#prediction using progressive net
pred = model_context.predict([padded_docs_prem_[35000:,:],padded_docs_hypo_[35000:,:]])

Executing op TensorDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op MapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op TensorDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Pack in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op FlatMapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op TensorDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op RepeatDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ZipDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ParallelMapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ModelDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op AnonymousIteratorV2 in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op __inference_distributed_function_83945 in device /job:localhost/replica:0/task:0/device:G

In [21]:
accuracy_score(np.argmax(pred[0],axis = 1),np.argmax(lab[35000:,:],axis = 1))

0.5712284913965586

In [22]:
model1 = MyModel1(30,max_len_multi_prem_,max_len_multi_hypo_,vocab_size_prem_,vocab_size_hypo_)
model1.compile(optimizer='adam',
                loss='categorical_crossentropy',
                metrics=['accuracy'])
model1.fit([padded_docs_prem_[0:35000,:],padded_docs_hypo_[0:35000,:]],[np.asarray(lab[0:35000,:],dtype=np.int32)],epochs = 2)

(?, 95, 30)
Executing op MapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op MapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op FlatMapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op TensorDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op RepeatDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ZipDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ParallelMapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Train on 35000 samples
Epoch 1/2
Executing op ModelDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op AnonymousIteratorV2 in device /job:localhost/replica:0/task:0/device:CPU:0
(?, 95, 30)
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:loc

In [23]:
x = model1.lstm_shared(model1.embedding1(padded_docs_prem_))
y = model1.lstm_shared(model1.embedding2(padded_docs_hypo_))
z = tf.math.abs(x-y)
print(z)
#print(np.sum(z_,axis = 1))
#print(np.sum(tf.keras.activations.softmax(z_),axis = 1))
z_ = model1.d1(z)
fin = model1.d2(z_)
from sklearn.metrics import accuracy_score
print(accuracy_score(np.argmax(fin[35000:,:],axis = 1),np.argmax(lab[35000:,:],axis = 1)))

Executing op ResourceGather in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Unpack in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Pack in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Unpack in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Pack in device /job:localhost/replica:0/task:0/device:GPU:0
tf.Tensor(
[[0.35192475 0.20617585 0.04888998 ... 0.8513858  1.4643469  0.08430769]
 [0.04488426 0.26570666 0.33149734 ... 0.55467683 1.4643538  0.08430769]
 [0.58657074 0.5699995  0.8748139  ... 0.8955618  1.4517334  0.08430769]
 ...
 [0.4854421  0.35768098 0.55506766 ... 0.01938966 1.468472   0.08650975]
 [0.8072926  0.4685302  0.39187786 ... 0.34041148 1.4684724  0.08650975]
 [0.7302002  0.54045737 0.52030456 ... 0.20773755 1.4682944  0.0690666 ]], shape=(39998, 40), dtype=float32)
0.5680272108843537
